In [ ]:
import pandas as pd
import tweepy
import json
import requests
from datetime import datetime
import matplotlib.pyplot as plt
from config import consumer_key,consumer_secret,access_token,access_token_secret
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
target_user = "@plotbotBot"
sentiments = []
counter = 1
tweet_id_all = []

In [ ]:

public_tweet = api.search(target_user,count = 1,search_type = "recent")


for tweet in public_tweet["statuses"]:
    tweet_content = tweet["text"]
    tweet_split = tweet_content.split(":")
    #print(tweet_split)
    sentiment_analyze = tweet_split[1]
    print(sentiment_analyze)

    for x in range(10):

        analyze_tweet = api.user_timeline(sentiment_analyze, page = x+1)

        for user_tweet in analyze_tweet:


            compound = analyzer.polarity_scores(user_tweet["text"])["compound"]
            pos = analyzer.polarity_scores(user_tweet["text"])["pos"]
            neu =  analyzer.polarity_scores(user_tweet["text"])["neu"]
            neg = analyzer.polarity_scores(user_tweet["text"])["neg"]

            

            sentiments.append({"user":user_tweet["user"]["name"],"Compound": compound,
                               "Positive": pos,
                               "Negative": neu,
                               "Neutral": neg, "Tweets Ago":counter})
            counter = counter + 1




In [ ]:
analyzed_df = pd.DataFrame.from_dict(sentiments)
analyzed_df.head()

In [ ]:
plt.plot(analyzed_df["Tweets Ago"],analyzed_df["Compound"],color="skyblue", marker="o", linewidth=0.5,
     alpha=0.8)
plt.xlabel("Tweets Ago")
plt.ylabel("Tweet Polarity")
plt.title(" As Per analysis on %s" % tweet["created_at"])
plt.savefig("plot.png")
plt.show()

In [ ]:
api.update_with_media("plot.png", "Thank you @%s! for the request on %s" % 
            (tweet["user"]["screen_name"],sentiment_analyze),
            in_reply_to_status_id=tweet["id"])
    tweet_id_all.append(tweet_id)